<div class="text_cell_render rendered_html" tabindex="-1">
    <p><img src="https://datascientest.fr/train/assets/logo_datascientest.png" style="height: 150px"></p>
    <p></p><hr style="border-width: 2px ; border-color: #75dfc1"><p></p>
    <p></p><h1 style="text-align: center" id="-Etude-d-un-modèle-"> Etude d'un modèle <a class="anchor-link" href="#-Etude-d-un-modèle-">¶</a></h1><p></p>
    <p></p><h2 style="text-align: center" id="-Introduction-"> Introduction <a class="anchor-link" href="#-Introduction-">¶</a></h2><p></p>
    <hr style="border-width: 2px ; border-color: #75dfc1">
    <blockquote>
        <p>Dans ce note book, on se propose de tester et valider les preformances d'un CNN.</p>
        <p>On peut céer un modèle personnalisé en utilisant la fonction model_libre du fichier modelPredefini.py, ou on peut  choisir parmis les modèles du fichier.</p><br><br>
        <p>On va tester les performances du modèle sur des images en niveaux de gris, puis en couleur avec 4 classes.</p>
    </blockquote>
    
</div>

# Première Partie - Préparation de l'environnement

## 1 - Importation des différentes librairies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix

from keras.optimizers import Adam

import tensorflow as tf

from keras.backend import sigmoid

# On importe nos librairies
import library.imageFunctions as mylib
import library.modelPredefini as mpf

## 2 - Fonctions d'activation complémentaires

### a - Fonction d'activation Mish
On définit une fonction d'activation pour des tests éventuels. Optionnel.

<h3 style="text-align: left" id="-Courte présentation-"> Courte présentation <a class="anchor-link" href="#-Introduction-">¶</a></h3>
<h6 style="text-align: left" id="-Extrait d'article-"> Extrait d'article : <a class="anchor-link" href="#-Introduction-">¶</a></h6><a href="https://penseeartificielle.fr/mish-vs-relu-meilleure-fonction-activation/" target="_blank" rel="noreferrer noopener" aria-label="Arxiv (opens in a new tab)">Article original</a>.</p>
<p>Mish est présentée comme une <em>fonction auto-régularisée, non-monotone</em>. Elle est inspirée des travaux de l’équipe de <strong>Google Brain</strong> qui a créé “<strong>Swish</strong>” en 2017, une autre fonction d’activation légèrement différente (et moins performante, d’où un article sur Mish) ! Mish a été inventée et codée par le mathématicien <strong>Diganta Misra</strong> </p>
<p>Voici le graphe de la fonction Mish :</p>
<div class="wp-block-image"><figure class="aligncenter is-resized"><img src="https://penseeartificielle.fr/wp-content/uploads/2019/10/mish-activation.png" alt="" class="wp-image-2418 td-animation-stack-type0-2" width="334" height="326" srcset="https://penseeartificielle.fr/wp-content/uploads/2019/10/mish-activation.png 579w, https://penseeartificielle.fr/wp-content/uploads/2019/10/mish-activation-300x293.png 300w, https://penseeartificielle.fr/wp-content/uploads/2019/10/mish-activation-430x420.png 430w" sizes="(max-width: 334px) 100vw, 334px"><figcaption>Graphe de la fonction Mish, qui se rapproche de 0 dans les valeurs négatives et tend vers l’infini dans les valeurs positives (crédit : Diganta Misra)</figcaption></figure></div>
<p>Mathématiquement parlant, Mish est une savante combinaison de tangente hyperbolique (vue précédemment), du logarithme népérien (“$ln$”) et de l’exponentielle (“$e$”) :  $mish(x)=x⋅tanh(ln((1+e^{x}))$</p>
<p>A noter que  $ln(1+e^{x})$  est souvent appelée fonction “softplus“, transformant la définition de Mish en  $mish(x)=x⋅tanh(softplus(x))$</p>

In [ ]:
def mish(inputs):
    return inputs * tf.math.tanh(tf.math.softplus(inputs))

### b - Fonction d'activation switch

<p>Tout d’abord, <a href="https://medium.com/@neuralnets/swish-activation-function-by-google-53e1ea86f820" target="_blank" rel="noreferrer noopener" aria-label="swish  (opens in a new tab)">swish </a>(qui est β-swish avec β=1) est définie ainsi :</p>
<span class="MathJax" id="MathJax-Element-6-Frame" tabindex="0" style=""><nobr><span class="math" id="MathJax-Span-124" style="width: 17.544em; display: inline-block;"><span style="display: inline-block; position: relative; width: 14.374em; height: 0px; font-size: 122%;"><span style="position: absolute; clip: rect(1.26em, 1014.26em, 2.626em, -999.997em); top: -2.183em; left: 0em;"><span class="mrow" id="MathJax-Span-125"><span class="mi" id="MathJax-Span-126" style="font-family: MathJax_Math; font-style: italic;">β<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mtext" id="MathJax-Span-127" style="font-family: MathJax_Main;">-swish</span><span class="mo" id="MathJax-Span-128" style="font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-129" style="font-family: MathJax_Math; font-style: italic;">x</span><span class="mo" id="MathJax-Span-130" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-131" style="font-family: MathJax_Math; font-style: italic; padding-left: 0.167em;">β<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-132" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-133" style="font-family: MathJax_Main; padding-left: 0.276em;">=</span><span class="mi" id="MathJax-Span-134" style="font-family: MathJax_Math; font-style: italic; padding-left: 0.276em;">x</span><span class="mo" id="MathJax-Span-135" style="font-family: MathJax_Main; padding-left: 0.221em;">⋅</span><span class="mi" id="MathJax-Span-136" style="font-family: MathJax_Math; font-style: italic; padding-left: 0.221em;">s</span><span class="mi" id="MathJax-Span-137" style="font-family: MathJax_Math; font-style: italic;">i</span><span class="mi" id="MathJax-Span-138" style="font-family: MathJax_Math; font-style: italic;">g<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mi" id="MathJax-Span-139" style="font-family: MathJax_Math; font-style: italic;">m</span><span class="mi" id="MathJax-Span-140" style="font-family: MathJax_Math; font-style: italic;">o</span><span class="mi" id="MathJax-Span-141" style="font-family: MathJax_Math; font-style: italic;">i</span><span class="mi" id="MathJax-Span-142" style="font-family: MathJax_Math; font-style: italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-143" style="font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-144" style="font-family: MathJax_Math; font-style: italic;">x</span><span class="mo" id="MathJax-Span-145" style="font-family: MathJax_Main; padding-left: 0.221em;">⋅</span><span class="mi" id="MathJax-Span-146" style="font-family: MathJax_Math; font-style: italic; padding-left: 0.221em;">β<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-147" style="font-family: MathJax_Main;">)</span></span><span style="display: inline-block; width: 0px; height: 2.189em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.397em; border-left: 0px solid; width: 0px; height: 1.337em;"></span></span></nobr></span>
<p>Mish et swish donnent quasiment la même courbe, avec quelques différences au niveau de la partie négative (on a une concavité plus importante avec mish) et une pente plus forte de la partie positive pour la fonction mish.</p>
<div class="wp-block-column">
<div class="wp-block-image"><figure class="aligncenter is-resized"><img src="https://penseeartificielle.fr/wp-content/uploads/2019/10/swish-activation.png" alt="" class="wp-image-2419 td-animation-stack-type0-2" width="566" height="146" srcset="https://penseeartificielle.fr/wp-content/uploads/2019/10/swish-activation.png 626w, https://penseeartificielle.fr/wp-content/uploads/2019/10/swish-activation-300x130.png 300w" sizes="(max-width: 283px) 100vw, 283px"><figcaption>Swish (crédit : Diganta Misra) </figcaption></figure></div>
</div>

In [ ]:
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

## 3 - Préparation des données pour les tests

### a - Les données

Ici, on commence à préparer les données pours les tests.    
On va donc faire une première préparation de base, qui va nous permettre en suite, de créer le jeux pour l'étude sur 4 classes et sur 15 classes.

In [ ]:
## Répertoires contenant les images
repTrain  = 'train_images/'
repTest   = 'test_images/'

In [ ]:
## Chargement du fichier train.csv
train = pd.read_csv('train.csv')

## Séparation nom de fichier / label
sep = '_'
train['image'] = train['Image_Label'].apply(lambda x: x.split(sep)[0])
train['label'] = train['Image_Label'].apply(lambda x: x.split(sep)[1])
train = train.drop(['Image_Label'], axis=1)

## Suppression des labels sans zone identifiées (variable EncodedPixels a NaN)
train = train.dropna()

## One Hot Encoding des classes de nuages et suprression des variables label et EncodedPixels
## On effectue une dichotomisation des classes de nuages
train_encoded = train
train_encoded= train_encoded.join(pd.get_dummies(train_encoded['label']))

## On supprime ensuite les colonnes "encoded Pixels" et "label", inutiles pour l'aggregation qui suit
train_encoded = train_encoded.drop(['EncodedPixels', 'label'], axis=1)

## Enfin on fait un sous-total pour ne conserver qu'une ligne par image
train_encoded = train_encoded.groupby(['image']).sum()

train_encoded = train_encoded.reset_index(level=0)
train_encoded.head()

Ici, on peut utiliser train_encoded tel quel, il nous faut juste définir la liste des classes pour la génération d'images.

In [ ]:
classes = ['Fish', 'Flower', 'Gravel', 'Sugar']

Maintenant, on va séparer le jeux en deux. Un pour l'entrainement, et qui sera utilisé dans le générateur et lautre qui ne servira que pour le test du modèle et pour les analysees.

In [ ]:
## On prend 20 % pour le test.
## On fixe le random_state à 42 pour utiliser le même jeux de données quelques soit les tests
train_encoded_train, train_encoded_test = train_test_split(train_encoded, 
                                                           test_size = 0.2, 
                                                           random_state = 42)

# Seconde Partie - Les tests

In [ ]:
# On définit nos variables globales
batch_size = 32
image_width = 140
image_height = 210


### A - Test en niveaux de gris

#### A.1. - Génération des images pour l'entrainement

In [ ]:
## On appelle la fonction qui génére les jeux
train_generator, validation_generator = mylib.image_data_generator(data=train_encoded_train,
                                                                  directory=repTrain,
                                                                  filename='image',
                                                                  classes=classes,
                                                                  image_width=image_width,
                                                                  image_height=image_height,
                                                                  canal=1,
                                                                  batch_size=batch_size,
                                                                  seed=42)   

In [ ]:
## On affiche la premiere image, qui peut changer a chaque execution puisque nous avons effectue un shuffle
## La premiere image est affichee ainsi que les augmentations qu'elle a subit
augmented_images = [train_generator[0][0][0] for i in range(5)]
mylib.plotImages(images_arr=augmented_images,
                canal=1)

#### A.2. - Entraînement du modèle

In [ ]:
## On crée le modèle avec des paramètres personnalisés
modele_libre = mpf.model_libre(image_width=image_width,
                               image_height=image_height,
                               nb_classes=4,
                               canaux=1,
                               coucheConv2D=1,
                               coucheDense=1,
                               GlobalAveragePooling2D_layer=False,
                               nb_filters_start=32,
                               kernel_size_start=(3,3),
                               activation_start='relu',
                               pool_size_start=(2,2),
                               padding_start='valid',
                               nb_filters=16,
                               kernel_size=(3,3),
                               pool_size=(2,2),
                               padding='valid',
                               nb_units=64,
                               activation='relu',
                               coeff_drop=0.2,
                               activation_sortie='sigmoid',
                               affichage = True)

In [ ]:
## On compile le modèle
lr = 1e-3
modele_libre.compile(optimizer=Adam(lr), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# On crée nos callback 
checkpoint = mpf.checkpoint_function(filepath=f"checkpoint/weights_modele_libre.hdf5")
lr_plateau = mpf.reduceLR_function()
e_stopping = mpf.estopping_function()

In [ ]:
# On entraine le modèle sur 50 époques
EPOCHS = 100
STEP_SIZE_TRAIN=train_generator.n//batch_size
STEP_SIZE_VALID=validation_generator.n//batch_size

history = modele_libre.fit_generator(
    train_generator,
    steps_per_epoch=STEP_SIZE_TRAIN,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=STEP_SIZE_VALID,
    callbacks=[lr_plateau, checkpoint, e_stopping])

#### A.3. - Evaluation du modèle

On commence par afficher les courbes d'évolution de l'accuracy et de la loss.

In [ ]:
## On fait appel à la fonction qui affiche la=es courbess.
epochTrain = len(history.history['loss'])
mylib.affiche_resultat(history, epochTrain, len(classes))

Maintenant, on va faire une prédiction sur le jeux de test.

In [ ]:
## On prépare le jeux de test avec la fonction convert_image(X, canaux, repPath):
X_test = train_encoded_test['image']
y_test = train_encoded_test[classes]

X_test_img = mylib.convert_image(data=X_test,
                                 image_width=image_width,
                                 image_height=image_height,
                                 canaux=1, 
                                 repPath=repTrain)

In [ ]:
## On fait des prédictions
y_pred = modele_libre.predict(X_test_img).round()


In [ ]:
## On affiche le rpport de classification
print(classification_report(y_test, y_pred, target_names = classes))

In [ ]:
## On affiche le score et la loss du modèle
scores = modele_libre.evaluate(X_test_img, y_test, verbose=0)
print("%s: %.2f" % (modele_libre.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (modele_libre.metrics_names[1], scores[1]*100))

In [ ]:
##On affiche la table de confusion
## Pour afficher la matrice de confusion on transforme y_pred
y_pred_df = pd.DataFrame(y_pred, index = y_test.index, columns = classes)

### Affichage de la matrice de confusion avec multilabel_confusion_matrix
cnf_matrix = multilabel_confusion_matrix(y_test, y_pred_df)
print(cnf_matrix)


### B - Test en couleurs

#### B.1. - Génération des images pour l'entrainement

In [ ]:
## On appelle la fonction qui génére les jeux
train_generator, validation_generator = mylib.image_data_generator(data=train_encoded_train,
                                                                  directory=repTrain,
                                                                  filename='image',
                                                                  classes=classes,
                                                                  image_width=image_width,
                                                                  image_height=image_height,
                                                                  canal=3,
                                                                  batch_size=batch_size,
                                                                  seed=42)   

In [ ]:
## On affiche la premiere image, qui peut changer a chaque execution puisque nous avons effectue un shuffle
## La premiere image est affichee ainsi que les augmentations qu'elle a subit
augmented_images = [train_generator[0][0][0] for i in range(5)]
mylib.plotImages(augmented_images, 3)

#### B.2. - Entraînement du modèle

In [ ]:
## On crée le modèle avec des paramètres personnalisés
modele_libre_color = mpf.model_libre(image_width=image_width,
                               image_height=image_height,
                               nb_classes=4,
                               canaux=3,
                               coucheConv2D=1,
                               coucheDense=1,
                               GlobalAveragePooling2D_layer=False,
                               nb_filters_start=32,
                               kernel_size_start=(3,3),
                               activation_start='relu',
                               pool_size_start=(2,2),
                               padding_start='valid',
                               nb_filters=16,
                               kernel_size=(3,3),
                               pool_size=(2,2),
                               padding='valid',
                               nb_units=64,
                               activation='relu',
                               coeff_drop=0.2,
                               activation_sortie='sigmoid',
                               affichage = True)

In [ ]:
## On compile le modèle
lr = 1e-3
modele_libre_color.compile(optimizer=Adam(lr), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# On crée nos callback 
checkpoint = mpf.checkpoint_function(filepath=f"checkpoint/weights_modele_libre_color.hdf5")
lr_plateau = mpf.reduceLR_function()
e_stopping = mpf.estopping_function()

In [ ]:
# On définit la variable contenant le chemin de sauvegarde du modèle
checkpointPath = 'checkpoint/modele_libre_color'

# On entraine le modèle sur 50 époques
EPOCHS = 100
STEP_SIZE_TRAIN=train_generator.n//batch_size
STEP_SIZE_VALID=validation_generator.n//batch_size

history = modele_libre_color.fit_generator(
    train_generator,
    steps_per_epoch=STEP_SIZE_TRAIN,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=STEP_SIZE_VALID,
    callbacks=[lr_plateau, checkpoint, e_stopping])

#### B.3. - Evaluation du modèle

In [ ]:
## On fait appel à la fonction qui affiche la=es courbess.
epochTrain = len(history.history['loss'])
mylib.affiche_resultat(history, epochTrain, len(classes))

In [ ]:
## On prépare le jeux de test avec la fonction convert_image(X, canaux, repPath):
X_test = train_encoded_test['image']
y_test = train_encoded_test[classes]

X_test_img_color = mylib.convert_image(data=X_test,
                                 image_width=image_width,
                                 image_height=image_height,
                                 canaux=3, 
                                 repPath=repTrain)

In [ ]:
## On fait des prédictions
y_pred_color = modele_libre_color.predict(X_test_img_color).round()

In [ ]:
## On affiche le rpport de classification
print(classification_report(y_test, y_pred_color, target_names = classes))

In [ ]:
## On affiche le score et la loss du modèle
scores = modele_libre_color.evaluate(X_test_img_color, y_test, verbose=0)
print("%s: %.2f" % (modele_libre_color.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (modele_libre_color.metrics_names[1], scores[1]*100))

In [ ]:
##On affiche la table de confusion
## Pour afficher la matrice de confusion on transforme y_pred
y_pred_color_df = pd.DataFrame(y_pred_color, index = y_test.index, columns = classes)

### Affichage de la matrice de confusion avec multilabel_confusion_matrix
cnf_matrix = multilabel_confusion_matrix(y_test, y_pred_color_df)
print(cnf_matrix)